# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [74]:
# install gluonts (most recent version)
!pip install git+https://github.com/awslabs/gluon-ts.git

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-wsjwy6sf
  Installing build dependencies ... done
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-kue7zl_c/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
Successfully built gluonts
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [75]:
# imports
import pandas as pd

import pprint
from functools import partial

# gluonts imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

# gluonts models
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator


In [76]:
# gluonts version (important)
import gluonts
print(gluonts.__version__)

0.3.4.dev79+g481fc5f


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [77]:
epochs = 100
num_batches_per_epoch = 200

datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]
print(datasets)

['m4_quarterly', 'm4_yearly']


In [78]:
estimators = [
#     partial(
#         SimpleFeedForwardEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
#     partial(
#         DeepAREstimator,
#         distr_output=PiecewiseLinearOutput(8),
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


INFO:root:Using CPU


In [79]:
def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

In [80]:
datasets

['m4_quarterly', 'm4_yearly']

In [81]:
import numpy as np
import mxnet as mx

In [82]:
if __name__ == "__main__":
    
    mx.random.seed(42)
    np.random.seed(42)
    
    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            print(estimator)
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly.
INFO:root:Start model training


functools.partial(<class 'gluonts.model.deepar._estimator.DeepAREstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08))
evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08), use_feat_dynamic_real=Fal

INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.00it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[0] Elapsed time 10.003 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.682601
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.36it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[1] Elapsed time 10.335 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.187988
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.90it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[2] Elapsed time 10.055 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.209194
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.17it/s, avg_epoch_loss=7.11]
INFO:root:Epoch[3] Elapsed time 10.439 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.109260
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:10<00:00, 19.66it/s, avg_epoch_loss=6.97]
INFO:root:E

{'Coverage[0.1]': 0.12719791666666666,
 'Coverage[0.2]': 0.23547916666666666,
 'Coverage[0.3]': 0.3465625,
 'Coverage[0.4]': 0.45766666666666667,
 'Coverage[0.5]': 0.5669270833333333,
 'Coverage[0.6]': 0.65740625,
 'Coverage[0.7]': 0.7489479166666667,
 'Coverage[0.8]': 0.8327239583333333,
 'Coverage[0.9]': 0.90846875,
 'MAE_Coverage': 0.042375578703703703,
 'MASE': 1.1861513502573564,
 'MSE': 1765542.6399753932,
 'MSIS': 12.653754518857925,
 'ND': 0.09403399785384435,
 'NRMSE': 0.22240721468038516,
 'QuantileLoss[0.1]': 49849717.43188706,
 'QuantileLoss[0.2]': 75919951.68797913,
 'QuantileLoss[0.3]': 93003162.64374237,
 'QuantileLoss[0.4]': 103320982.27595215,
 'QuantileLoss[0.5]': 107863960.7726059,
 'QuantileLoss[0.6]': 106462717.99465942,
 'QuantileLoss[0.7]': 99262580.98280334,
 'QuantileLoss[0.8]': 84727111.27318114,
 'QuantileLoss[0.9]': 59598954.38851623,
 'RMSE': 1328.7372351128695,
 'abs_error': 107863960.72151184,
 'abs_target_mean': 5974.344119287491,
 'abs_target_sum': 1147

100%|██████████| 200/200 [00:08<00:00, 23.23it/s, avg_epoch_loss=7.5]
INFO:root:Epoch[0] Elapsed time 8.617 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.502066
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:08<00:00, 22.49it/s, avg_epoch_loss=7.04]
INFO:root:Epoch[1] Elapsed time 8.895 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.040929
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:13<00:00, 14.70it/s, avg_epoch_loss=7.1] 
INFO:root:Epoch[2] Elapsed time 13.611 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.103089
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:34<00:00,  5.78it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[3] Elapsed time 34.596 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.972173
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:28<00:00,  7.09it/s, avg_epoch_loss=6.26]
INFO:root:Epoch[4] Elapsed time 28.230 seconds
INFO:root

KeyboardInterrupt: 

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS","wQuantileLoss[0.5]", "wQuantileLoss[0.9]", ]]